In [14]:
%matplotlib inline
# Importing libraries
import matplotlib.pyplot as plt
import pandas as pd

FILEPATH_PREFIX = '../../../book_crawler/data'
SPIDERNAME = 'bookoutlet'
FILENAME = '2019-05-12T14-37-10.csv'
FILEPATH = '{}/{}/{}'.format(FILEPATH_PREFIX, SPIDERNAME, FILENAME)
FILEPATH

'../../../book_crawler/data/bookoutlet/2019-05-12T14-37-10.csv'

In [15]:
df = pd.read_csv(FILEPATH)

## Data analisys
Some preliminary analisys of the dataset

In [16]:
print("First 5 rows")
print("------------")
df.head()

First 5 rows
------------


,url,referer_url,src
0,https://bookoutlet.com/,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\..."
1,https://bookoutlet.com/Store/Sale,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\..."
2,https://bookoutlet.com/Store/OtherBrowsing,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\..."
3,https://bookoutlet.com/Store/Browse?N=isTopTen...,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\..."
4,https://bookoutlet.com/Store/Browse?N=isGiftCe...,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\..."


In [17]:
print("No. of rows and columns")
print("-----------------------")
df.shape

No. of rows and columns
-----------------------


(16387, 3)

In [18]:
print("Check null values")
print("-----------------")
df.isnull().any().any()

Check null values
-----------------


False

In [19]:
print("Check duplicate values")
print("----------------------")
len(df['url'].unique()) != df.shape[0]

Check duplicate values
----------------------


False

In [20]:
print("DataFrame column types")
print("----------------------")
df.info()

DataFrame column types
----------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16387 entries, 0 to 16386
Data columns (total 3 columns):
url            16387 non-null object
referer_url    16387 non-null object
src            16387 non-null object
dtypes: object(3)
memory usage: 384.1+ KB


In [21]:
print("Some stats")
print("----------------")
df.describe()

Some stats
----------------


,url,referer_url,src
count,16387,16387,16387
unique,16387,8050,16387
top,https://bookoutlet.com/Store/Browse?Na=234405&...,https://bookoutlet.com/Store/Browse?Npb=6326,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\..."
freq,1,31,1


## Data cleaning
During the crawling phase (using Scrapy), we restrained the crawled pages to be only the web pages matching with `https://bookoutlet.com/` domain.

In [27]:
import re

In [28]:
bookoutletRegex = r'^https\:\/\/bookoutlet\.com\/'

In [29]:
otherDomainsDataFrame = df[- df['url'].str.match(bookoutletRegex)]

In [30]:
otherDomainsDataFrame.head()

,url,referer_url,src


So Scrapy effectively crawled only pages from the `www.bookoutlet.com` domain

## Add `shingle_vector` label
Compute shingle vector for each page of the dataframe

In [32]:
#add top level folder to sys.path
import sys
sys.path.append('../../../')

from foxlink_clustering.clustering.shingler import compute_shingle_vector

In [34]:
src = df.iloc[0]['src']

The default value for window size in Foxlink is 3

In [35]:
DEFAULT_WINDOW_SIZE = 3
result = compute_shingle_vector(src, DEFAULT_WINDOW_SIZE)

In [36]:
result

(0, 3, 1, 4, 0, 1, 3, 0)

Set `shingle_vector` label for each row    

In [37]:
df['shingle_vector'] = df.apply(lambda x: compute_shingle_vector(x['src'], DEFAULT_WINDOW_SIZE), axis=1)

In [38]:
df.head()

,url,referer_url,src,shingle_vector
0,https://bookoutlet.com/,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...","(0, 3, 1, 4, 0, 1, 3, 0)"
1,https://bookoutlet.com/Store/Sale,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...","(0, 3, 1, 0, 0, 1, 3, 0)"
2,https://bookoutlet.com/Store/OtherBrowsing,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...","(0, 3, 1, 8, 1, 1, 1, 0)"
3,https://bookoutlet.com/Store/Browse?N=isTopTen...,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...","(0, 3, 1, 1, 1, 1, 0, 0)"
4,https://bookoutlet.com/Store/Browse?N=isGiftCe...,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...","(0, 3, 1, 1, 1, 1, 0, 0)"


## Write dataset to a file

In [39]:
df.to_csv('../../../datasets/bookoutlet.csv', encoding='utf-8', index=False)

In [40]:
test = pd.read_csv('../../../datasets/bookoutlet.csv')

In [41]:
test.head()

,url,referer_url,src,shingle_vector
0,https://bookoutlet.com/,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...","(0, 3, 1, 4, 0, 1, 3, 0)"
1,https://bookoutlet.com/Store/Sale,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...","(0, 3, 1, 0, 0, 1, 3, 0)"
2,https://bookoutlet.com/Store/OtherBrowsing,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...","(0, 3, 1, 8, 1, 1, 1, 0)"
3,https://bookoutlet.com/Store/Browse?N=isTopTen...,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...","(0, 3, 1, 1, 1, 1, 0, 0)"
4,https://bookoutlet.com/Store/Browse?N=isGiftCe...,https://bookoutlet.com/,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...","(0, 3, 1, 1, 1, 1, 0, 0)"


In [42]:
test.shape

(16387, 4)